In [34]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as sqlf 
from pyspark.sql import DataFrameReader, SQLContext
#col, lit, udf,sum,avg,max,min,mean,count, udf 
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType, LongType

In [35]:
spark = SparkSession.builder.config('spark.jars', '/Users/gonzo/Downloads/postgresql-42.5.0.jar').getOrCreate()
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

22/10/21 09:34:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/21 09:34:49 WARN DependencyUtils: Local jar /Users/gonzo/Downloads/postgresql-42.5.0.jar does not exist, skipping.
22/10/21 09:34:49 INFO SparkContext: Running Spark version 3.3.0
22/10/21 09:34:49 INFO ResourceUtils: ==============================================================
22/10/21 09:34:49 INFO ResourceUtils: No custom resources configured for spark.driver.
22/10/21 09:34:49 INFO ResourceUtils: ==============================================================
22/10/21 09:34:49 INFO SparkContext: Submitted application: pyspark-shell
22/10/21 09:34:49 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, am

/opt/anaconda3/envs/CapstoneENV/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [39]:
sc.stop()

22/10/21 11:09:38 INFO SparkUI: Stopped Spark web UI at http://gonzalos-mbp.griddynamics.com:4040
22/10/21 11:09:39 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
22/10/21 11:09:39 INFO MemoryStore: MemoryStore cleared
22/10/21 11:09:39 INFO BlockManager: BlockManager stopped
22/10/21 11:09:39 INFO BlockManagerMaster: BlockManagerMaster stopped
22/10/21 11:09:39 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
22/10/21 11:09:39 INFO SparkContext: Successfully stopped SparkContext


In [36]:
import json
import logging
from pandas import DataFrame
import os
import pendulum
from datetime import datetime

def get_subdirectories(path: str) -> list:
    """
    Function that searchs for sub-directories inside a directory 
    then returns a list of all the directories names.
    """
    if path.split('/')[7] in ['customer', 'transaction']:
        subdirectories = [f.path for f in os.scandir(path) if f.is_dir()]
        subdirectories = [x.split('/')[8] for x in subdirectories]
        csv_directories = [directory.split('_')[1] for directory in subdirectories]
        return csv_directories
    else:
        subdirectories = [f.path for f in os.scandir(path) if f.is_dir()]
        subdirectories = [x.split('/')[8] for x in subdirectories]
        return [datetime.strptime(directory, '%Y-%m-%d').date() for directory in subdirectories] 
        

def get_latest_folder(folders: list) -> str:
    """
    Returns the latest date from the list.
    """
    try:
        return str(max(folders))
    except:
        return logging.critical('Couldn\'t find any sub-directory.')


def loadJsonData(json_path: str) -> DataFrame:
    """
    Function that returns a dataframe from a valid directory that contains jsonlines files.
    """
    latest = get_latest_folder(get_subdirectories(json_path))
    if os.path.exists(os.path.dirname(os.path.join(json_path, latest))):
        df = spark.read.json(os.path.join(json_path, latest))
        return df
    else:
        return logging.critical('Path to jsonl files doesn\'t exist')


def loadParquetData(parquet_path: str) -> DataFrame:
    """
    Function that returns a dataframe from a valid directory that contains parquet files.
    """
    latest = get_latest_folder(get_subdirectories(parquet_path))
    if os.path.exists(os.path.dirname(parquet_path)):
        df = spark.read.option('recursiveFileLookup', 'true').option('header', 'true').parquet(parquet_path)
        return df
    else:
        return logging.critical('Path to parquet file deosn\'t exist')


# def loadCSVData(csv_path: str) -> DataFrame:
#     """
#     Function that returns a dataframe from a valid directory that contains csv files.
#     """
#     latest = get_latest_folder(get_subdirectories(csv_path))
#     if os.path.exists(os.path.dirname(csv_path)):
#         df = spark.read.option('recursiveFileLookup', 'true').option('header', 'true').csv(csv_path)
#         return df
#     else:
#         return logging.critical('Path to csv file doesn\'t exist')

In [37]:
# Ingesting jsonlines data
jsonDF = loadJsonData(json_path='/Users/gonzo/Desktop/capstone_project/airflow/dags/json_storage/')
jsonDF = jsonDF.select('id', 'ts', 'customer_first_name', 'customer_last_name', 'amount', 'type')
jsonDF = jsonDF.repartition(2)
jsonDF.rdd.getNumPartitions()

22/10/21 09:34:52 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
22/10/21 09:34:52 INFO SharedState: Warehouse path is 'file:/Users/gonzo/Desktop/capstone_project/pyspark/jupiter%20notebooks/spark-warehouse'.
22/10/21 09:34:54 INFO InMemoryFileIndex: It took 43 ms to list leaf files for 1 paths.
22/10/21 09:34:54 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
22/10/21 09:34:57 INFO FileSourceStrategy: Pushed Filters: 
22/10/21 09:34:57 INFO FileSourceStrategy: Post-Scan Filters: 
22/10/21 09:34:57 INFO FileSourceStrategy: Output Data Schema: struct<value: string>
22/10/21 09:34:57 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 350.2 KiB, free 366.0 MiB)
22/10/21 09:34:58 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 365.9 MiB)
22/10/21 09:34:58 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on gonzalos-mbp.gridd

22/10/21 09:34:59 INFO CodeGenerator: Code generated in 330.396496 ms
22/10/21 09:34:59 INFO Executor: Finished task 0.0 in stage 0.0 (TID 0). 2144 bytes result sent to driver
22/10/21 09:34:59 INFO TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 869 ms on gonzalos-mbp.griddynamics.com (executor driver) (1/1)
22/10/21 09:34:59 INFO TaskSchedulerImpl: Removed TaskSet 0.0, whose tasks have all completed, from pool 
22/10/21 09:34:59 INFO DAGScheduler: ResultStage 0 (json at NativeMethodAccessorImpl.java:0) finished in 1.019 s
22/10/21 09:34:59 INFO DAGScheduler: Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
22/10/21 09:34:59 INFO TaskSchedulerImpl: Killing all running tasks in stage 0: Stage finished
22/10/21 09:34:59 INFO DAGScheduler: Job 0 finished: json at NativeMethodAccessorImpl.java:0, took 1.080717 s


22/10/21 09:34:59 INFO FileSourceStrategy: Pushed Filters: 
22/10/21 09:34:59 INFO FileSourceStrategy: Post-Scan Filters: 
22/10/21 09:34:59 INFO FileSourceStrategy: Output Data Schema: struct<amount: string, customer_first_name: string, customer_last_name: string, id: bigint, ts: string ... 1 more field>
22/10/21 09:35:00 INFO CodeGenerator: Code generated in 30.118872 ms
22/10/21 09:35:00 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 350.1 KiB, free 365.6 MiB)
22/10/21 09:35:00 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.5 MiB)
22/10/21 09:35:00 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on gonzalos-mbp.griddynamics.com:61635 (size: 34.1 KiB, free: 366.2 MiB)
22/10/21 09:35:00 INFO SparkContext: Created broadcast 2 from javaToPython at NativeMethodAccessorImpl.java:0
22/10/21 09:35:00 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is

2

In [33]:
# Ingesting parquet data
parquetDF = loadParquetData(parquet_path='/Users/gonzo/Desktop/capstone_project/airflow/dags/parquet_storage/')
parquetDF = parquetDF.repartition(2)
parquetDF.count()

NameError: name 'loadParquetData' is not defined

# Connection to PostgreSQL DB

In [14]:
url = 'jdbc:postgresql://192.168.16.4:5432/capstone'
properties = {'user': 'root', 'password': 'root'}
postgresql_df = sqlContext.read.format('jdbc').options(url=url, driver='org.postgresql.Driver', dbtable='customer', properties=properties).load()

Py4JJavaError: An error occurred while calling o105.load.
: org.postgresql.util.PSQLException: The connection attempt failed.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:331)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:49)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:247)
	at org.postgresql.Driver.makeConnection(Driver.java:434)
	at org.postgresql.Driver.connect(Driver.java:291)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:122)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:118)
	at org.apache.spark.sql.execution.datasources

In [12]:
postgresql_df.show()

+---+----------+---------+------------+-------+
| id|first_name|last_name|phone_number|address|
+---+----------+---------+------------+-------+
+---+----------+---------+------------+-------+



# Union between json and parquet DF's

In [887]:
parquetDF = parquetDF.withColumnRenamed('First_name', 'customer_first_name')
parquetDF = parquetDF.withColumnRenamed('Last_name', 'customer_last_name')
parquetDF = parquetDF.withColumnRenamed('Amount', 'amount')
jsonDF = jsonDF.withColumnRenamed('ts', 'timestamp')
jsonDF = jsonDF.withColumnRenamed('Store_id', 'store_id')

In [890]:
# Parquet & Json
json_parquetDF = jsonDF.join(parquetDF, ['id', 'customer_first_name', 'customer_last_name', 'amount', 'timestamp'], "fullouter")
json_parquetDF = json_parquetDF.select('id', 'type', 'store_id', 'amount', 'customer_first_name', 'customer_last_name', 'timestamp')
json_parquetDF = json_parquetDF.repartition(2)

In [607]:
# Both dataframes from postgresql
postgresqlDF = customerDF.join(transactionDF, ['id'], "inner")
postgresqlDF = postgresqlDF.select('id', 'customer_id', 'amount', 'first_name', 'last_name', 'phone_number', 'address', 'transaction_ts')
postgresqlDF.show(100)

+---+-----------+----------+-----------+----------+------------+-----------------+-------------------+
| id|customer_id|    amount| first_name| last_name|phone_number|          address|     transaction_ts|
+---+-----------+----------+-----------+----------+------------+-----------------+-------------------+
|778|         39| $8,278.52|    Cynthia|   Johnson|  3253956605|      Veronicaton|2022-10-03 15:09:37|
| 33|        191| $2,917.29|       John|     Burns|  8671875822|Port Benjaminfurt|2022-10-03 05:58:39|
|482|        106| $7,042.53|      Carol| Rodriguez|  9808399017|      Veronicaton|2022-10-03 03:48:50|
|609|        759|    $41.25|    Cynthia| Hernandez|  1825888381|       Robertland|2022-10-03 22:08:51|
|459|        358|   $709.30|       Leah|      Ward|  2449557321|      Lake Karina|2022-10-03 02:52:32|
|414|         52|    $50.11|      David|      Barr|  2388757413|      Veronicaton|2022-10-04 16:07:52|
|556|        808|     $6.85|      Kelly|     Smith|  3880545614|        S

In [684]:
postgresqlDF = postgresqlDF.withColumnRenamed('first_name', 'customer_first_name')
postgresqlDF = postgresqlDF.withColumnRenamed('last_name', 'customer_last_name')
postgresqlDF = postgresqlDF.withColumnRenamed('transaction_ts', 'timestamp')
"""
Change id str -> long
customer_id str -> long
timestamp remove ' ' 
"""
def transform_timestampPSQL(x) -> str:
    return x.split(' ')[0]

timestampPSQLUDF = sqlf.udf(lambda x : transform_timestampPSQL(x), StringType())

postgresqlDF = postgresqlDF.withColumn('id', postgresqlDF['id'].cast(LongType()))
postgresqlDF = postgresqlDF.withColumn('customer_id', postgresqlDF['id'].cast(LongType()))
postgresqlDF = postgresqlDF.withColumn('timestamp', timestampPSQLUDF(postgresqlDF.timestamp))
postgresqlDF.show()

+---+-----------+----------+-------------------+------------------+------------+-----------------+----------+
| id|customer_id|    amount|customer_first_name|customer_last_name|phone_number|          address| timestamp|
+---+-----------+----------+-------------------+------------------+------------+-----------------+----------+
|778|        778| $8,278.52|            Cynthia|           Johnson|  3253956605|      Veronicaton|2022-10-03|
| 33|         33| $2,917.29|               John|             Burns|  8671875822|Port Benjaminfurt|2022-10-03|
|482|        482| $7,042.53|              Carol|         Rodriguez|  9808399017|      Veronicaton|2022-10-03|
|609|        609|    $41.25|            Cynthia|         Hernandez|  1825888381|       Robertland|2022-10-03|
|459|        459|   $709.30|               Leah|              Ward|  2449557321|      Lake Karina|2022-10-03|
|414|        414|    $50.11|              David|              Barr|  2388757413|      Veronicaton|2022-10-04|
|556|     

In [685]:
unified_model = json_parquetDF.join(postgresqlDF, ['id', 'amount', 'customer_first_name', 'customer_last_name', 'timestamp'], 'fullouter')
unified_model = unified_model.select('id', 'customer_id', 'store_id', 'type', 'amount', 'customer_first_name', 'customer_last_name', 'phone_number', 'address', 'timestamp')
unified_model.count()

300

In [611]:

# unified_model.coalesce(1)
# unified_model = unified_model.repartition(2)
# unified_model.write.csv('/Users/gonzo/Desktop/capstone_project/data_storage/storage/test', header=True)

In [686]:
"""
Transform amount str -> float
"""

def transform_amount(x) -> list:
    return float(x[1:].replace(',', ''))


def transform_timestamp(x) -> str:
    return x.split('T')[0]

transformUDF = sqlf.udf(lambda x : transform_amount(x), FloatType())
timestampUDF = sqlf.udf(lambda x : transform_timestamp(x), StringType())

dumpDF = unified_model.withColumn('float_amount', transformUDF(unified_model.amount))
dumpDF = dumpDF.withColumn('timestamp', timestampUDF(dumpDF.timestamp))

In [687]:
unified_model = dumpDF.select('id', 'customer_id', 'store_id', 'type', 'float_amount', 'customer_first_name', 'customer_last_name', 'phone_number', 'address', 'timestamp')
unified_model = unified_model.withColumnRenamed('float_amount', 'amount')

In [708]:
unified_model.show()

+---+-----------+--------+----+--------+-------------------+------------------+------------+-----------------+----------+
| id|customer_id|store_id|type|  amount|customer_first_name|customer_last_name|phone_number|          address| timestamp|
+---+-----------+--------+----+--------+-------------------+------------------+------------+-----------------+----------+
|  2|       null|    null|   1|    4.18|             Brenda|           Sanchez|        null|             null|2022-10-02|
|  4|       null|    null|   0|23199.16|               Dawn|              Hill|        null|             null|2022-10-03|
|  6|       null|    null|   0|84143.57|            Jeffrey|          Gonzales|        null|             null|2022-10-04|
|  7|       null|      17|null|  711.35|             Thomas|            Oliver|        null|             null|2022-10-05|
| 10|       null|      20|null|    67.4|            William|             Reyes|        null|             null|2022-10-04|
| 11|       null|       

# Count and sum amount transactions for each type for day - online

In [726]:
online = unified_model.select('type', 'timestamp', 'amount')
online = online.filter(online.type == 1).groupBy(online.type, online.timestamp).agg(sqlf.count('*').alias('total_transactions'), sqlf.sum('amount').alias('total'))
online.sort('timestamp').show()

+----+----------+------------------+------------------+
|type| timestamp|total_transactions|             total|
+----+----------+------------------+------------------+
|   1|2022-10-02|                10|171386.61967229843|
|   1|2022-10-03|                10| 69591.97856712341|
|   1|2022-10-04|                17| 80872.64190080762|
|   1|2022-10-05|                 9|185163.73170089722|
+----+----------+------------------+------------------+



# Count and sum amount transactions for each type for day - offline

In [725]:
offline = unified_model.select('type', 'timestamp', 'amount')
offline = offline.filter(offline.type == 0).groupBy(offline.type, offline.timestamp).agg(sqlf.count('*').alias('total_transactions'), sqlf.sum('amount').alias('total'))
offline.sort('timestamp').show()

+----+----------+------------------+------------------+
|type| timestamp|total_transactions|             total|
+----+----------+------------------+------------------+
|   0|2022-10-02|                 6|1485.7499787807465|
|   0|2022-10-03|                16|179331.11799812317|
|   0|2022-10-04|                25|231126.67954114825|
|   0|2022-10-05|                 7| 4441.390130549669|
+----+----------+------------------+------------------+



# Count and sum transaction for each store for day

In [724]:
store_metrics  = unified_model.groupBy(unified_model.store_id, unified_model.timestamp).agg(sqlf.count(unified_model.timestamp).alias('count'), sqlf.sum(unified_model.amount).alias('total'))
store_metrics.sort('timestamp').show()

+--------+----------+-----+------------------+
|store_id| timestamp|count|             total|
+--------+----------+-----+------------------+
|    null|2022-10-02|   29| 425310.2145335674|
|      18|2022-10-02|    2| 782.8299789428711|
|      12|2022-10-02|    1|  98.4000015258789|
|       8|2022-10-02|    1|  68.2699966430664|
|       3|2022-10-02|    1|     66788.5703125|
|       5|2022-10-02|    1|  1369.56005859375|
|      17|2022-10-02|    3| 718.2699794769287|
|      10|2022-10-02|    1| 435.4100036621094|
|      20|2022-10-02|    1| 58.08000183105469|
|       7|2022-10-02|    1| 1110.449951171875|
|      14|2022-10-02|    2| 831.7099990844727|
|       6|2022-10-02|    2| 9549.819726467133|
|    null|2022-10-03|   63| 613245.9728010893|
|      18|2022-10-03|    3| 7946.419731140137|
|       9|2022-10-03|    2| 10383.47998046875|
|      12|2022-10-03|    4| 90181.04906272888|
|       8|2022-10-03|    3|183019.29092407227|
|       1|2022-10-03|    4| 84645.54937458038|
|       3|202

# Count and sum amount transactions for each city for day

In [731]:
cities_transactions = unified_model.groupBy(unified_model.address, unified_model.timestamp).agg(sqlf.count('*').alias('count'), sqlf.sum(unified_model.amount).alias('total'))
cities_transactions.sort('timestamp').show()

+-----------------+----------+-----+------------------+
|          address| timestamp|count|             total|
+-----------------+----------+-----+------------------+
|        Erikville|2022-10-02|    1|    8222.150390625|
|             null|2022-10-02|   32|254683.73966097832|
|     Brownchester|2022-10-02|    1|382.70001220703125|
|        Smithberg|2022-10-02|    1| 6.599999904632568|
|       Robertland|2022-10-02|    1|       49262.71875|
|Port Benjaminfurt|2022-10-02|    1|    65182.94921875|
|         Lanefurt|2022-10-02|    2|            197.25|
|       Grahamstad|2022-10-02|    1|    52058.44921875|
|         Clayview|2022-10-02|    1|  8151.77978515625|
|       Averymouth|2022-10-02|    2| 2983.210006713867|
|    Michelleburgh|2022-10-02|    2| 65990.03750038147|
|        Hicksview|2022-10-03|    2|  4112.47998046875|
|             null|2022-10-03|   53| 716727.3829708099|
|     Edwardsburgh|2022-10-03|    1|  9.40999984741211|
|      Melissafurt|2022-10-03|    5|  54707.2892

# BACKFILLING ALL THE NULL VALUES FROM THE COLUMNS

In [845]:
import random 
from faker import Faker

def backfillStoreID(x) -> int:
    if x is None:
        return random.randint(1,20)
    else:
        return x

def backfillPhoneNumber(x) -> int:
    fake = Faker()
    if x is None:
        return fake.msisdn()[3:]
    else:
        return x

def backfillAddress(x) -> str:
    if x is None:
        return random.choice(['Allenton', 'Hicksview', 'Smithberg', 'Robertland', 'Veronicaton', 'Lake Jamesville', 'Port Benjaminfurt', 'Averymouth', 'Erikville', 'Port Loriview', 'Grahamstad', 'Edwardsburgh', 'New Marthaborough', 'Melissafurt', 'Lanefurt', 'Clayview', 'West Nichole', 'Brownchester', 'Lake Karina', 'Michelleburgh'])
    else:
        return x

def backfillCustomerID(x) -> int:
    if x is None:
        return random.randint(1, 900)
    else:
        return x

backfillStoreId_udf = sqlf.udf(lambda x : backfillStoreID(x), IntegerType())
backfillPhoneNumber_udf = sqlf.udf(lambda x: backfillPhoneNumber(x), StringType())
backfillAddress_udf = sqlf.udf(lambda x: backfillAddress(x), StringType())
backfillCustomerID_udf = sqlf.udf(lambda x: backfillCustomerID(x), LongType())

In [777]:
testing_2 = unified_model
testing_2.repartition(2).rdd.getNumPartitions()

2

# Backfill Type

In [842]:
testing_2 = testing_2.withColumn('new_type', sqlf.when(testing_2.type.isNull(), sqlf.rand()).otherwise(testing_2.type))
testing_2 = testing_2.withColumn('b_type', sqlf.when(testing_2.new_type >= 0.5, 1).otherwise(0))

# Backfill Store ID

In [846]:
testing_2 = testing_2.withColumn('b_store_id', backfillStoreId_udf(testing_2.store_id))

# Backfill Phone Number

In [849]:
testing_2 = testing_2.withColumn('b_phone_number', backfillPhoneNumber_udf(testing_2.phone_number))

# Backfill Address

In [853]:
testing_2 = testing_2.withColumn('b_address', backfillAddress_udf(testing_2.address))

# Backfill Customer ID

In [856]:
testing_2 = testing_2.withColumn('b_customer_id', backfillCustomerID_udf(testing_2.customer_id))

In [858]:
backfilled_DF = testing_2.select('id', 'b_customer_id', 'customer_id', 'store_id', 'b_store_id', 'type', 'b_type', 'customer_first_name', 'customer_last_name', 'b_phone_number', 'phone_number', 'b_address', 'address', 'timestamp')

In [859]:
backfilled_DF.show()

+---+-------------+-----------+--------+----------+----+------+-------------------+------------------+--------------+------------+-----------------+-----------------+----------+
| id|b_customer_id|customer_id|store_id|b_store_id|type|b_type|customer_first_name|customer_last_name|b_phone_number|phone_number|        b_address|          address| timestamp|
+---+-------------+-----------+--------+----------+----+------+-------------------+------------------+--------------+------------+-----------------+-----------------+----------+
|  2|          327|       null|    null|         5|   1|     1|             Brenda|           Sanchez|    0283172417|        null|     Brownchester|             null|2022-10-02|
|  4|          502|       null|    null|         7|   0|     0|               Dawn|              Hill|    4808411130|        null|         Allenton|             null|2022-10-03|
|  6|          857|       null|    null|         2|   0|     0|            Jeffrey|          Gonzales|    5566

In [7]:
import pandas as pd

df = pd.read_parquet('/Users/gonzo/Desktop/capstone_project/airflow/dags/storage/rdbms_temp/')
print(df)

      id  type  store_id      amount customer_first_name customer_last_name  \
0    386   NaN      11.0       $5.04              Colton             Reeves   
1    417   NaN       6.0     $583.96               Scott            Roberts   
2    624   0.0       NaN     $206.94             Kristen             Porter   
3    324   NaN      12.0     $459.10             Michael              Gross   
4     47   1.0       NaN     $488.94              Sergio             Wilson   
..   ...   ...       ...         ...                 ...                ...   
195  166   1.0       NaN       $5.66               Karla             Duarte   
196  255   0.0       NaN   $3,428.23             Kenneth           Gonzalez   
197  125   1.0       NaN   $3,579.21             Timothy               Cruz   
198  692   NaN      16.0   $1,072.52            Brittany          Velazquez   
199  685   NaN      19.0  $34,941.97                Dawn              Smith   

               timestamp  
0    2022-10-18T04:05:07